In [ ]:
%matplotlib inline
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from dataset import NucleiDataset
from utils import *
from losses import focal_loss, dice_loss
from unet import UNet

We need to set the path of the data and wrap the data in a so-called dataloader, that will be supplying data samples to the network during the training process

In [ ]:
TRAIN_DATA_PATH = '/g/kreshuk/zinchenk/courses/EMBL_BTM_2019/advanced_machine_learning/nuclei_train_data'
train_data = NucleiDataset(TRAIN_DATA_PATH, RandomCrop(256))
train_dataloader = DataLoader(train_data)

Let's see how the data looks like. We have raw input images and ground truth - manually annotated nuclei

In [ ]:
show_dataset(train_data)

We also want to have some data not used for training to check how the model performs on data it hasn't seen before

In [ ]:
TEST_DATA_PATH = '/g/kreshuk/zinchenk/courses/EMBL_BTM_2019/advanced_machine_learning/nuclei_test_data'
test_data = NucleiDataset(TEST_DATA_PATH, RandomCrop(256))
test_dataloader = DataLoader(test_data)

In [ ]:
show_dataset(test_data)

And now we will load the model that we want to train. The architecture is called UNet and it has been steadily outperforming the other architectures in segmenting biological and medical images. This is how it looks like:

<img src="./imgs/u-net.png">

Now let's set the hyperparameters we would need further

In [ ]:
NUM_LAYERS = 3    # determines the capacity and the 'depth' (filed of view) of the network
IN_FILTERS = 32   # the number of the feature maps in the first layer - also affects the capacity

In [ ]:
model=UNet(IN_FILTERS, NUM_LAYERS)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
def train(model, optimizer, train_loader, test_loader, num_epochs=10):
    dataset_size = len(train_loader)   #how many samples we have
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        model.train()
        train_loss = 0.0    # the value that we will backprop
        train_accuracy = 0.0    # just a helpful (for us) metric
        train_iou = 0.0    # another helpful metric
        count = 0
        for images, masks in iterate(train_loader):
            count += 1
            optimizer.zero_grad()    # erase all the gradient from the previous steps
            outputs = model(images)    # predict
            predictions = (outputs > 0.5)    # binarize the predictions to get a mask
            if count % 10 == 0:    # every tenth iteration show how we are performing
                show_images(images, masks, predictions)
            loss = focal_loss(outputs, masks)    # calculate the loss between the predictions and the ground truth
            accuracy = torch.mean((predictions == masks.byte()).float())    # how much is the prediction similar to the ground truth? pixelwise
            iou = get_iou(predictions, masks.type(torch.bool))    # calculate the intersection over union (explained below)
            loss.backward()    # compute the gradients for every neuron
            optimizer.step()    # backpropagate!
            train_loss += loss.item()
            train_accuracy += accuracy.item()
            train_iou += iou.item()
        epoch_loss = train_loss / dataset_size
        epoch_accuracy = train_accuracy / dataset_size
        epoch_iou = train_iou / dataset_size
        print('Training loss is {:.6f}, iou is {:.6f}, accuracy is {:.6f}'.format(epoch_loss, epoch_iou, epoch_accuracy))
        evaluate(test_loader, model)    # every epoch we want to check how the model performs on a previously unseen data
    return model

Now we can train the model to see how the prediction accuracy changes over the training time

In [ ]:
model = train(model, optimizer, train_dataloader, test_dataloader, num_epochs=10)

This is how the IOU (intersection over union) works:
<img src="./imgs/iou1.png">
<img src="./imgs/iou2.png">